In [2]:
using Turing, Distributions, Random, MCMCChains, Plots, StatsPlots, Measures, BSON
#Load utility functions
include("utility_functions.jl")
# Random seed for reproducibility
Random.seed!(1234)

TaskLocalRNG()

In [3]:
# Define the Overhypothesis model using Turing
@model function dirichlet_multinomial_model(K, I, n, y)
    # Priors
    alpha ~ truncated(Exponential(0.5), 0.1, Inf)  # Keep alpha strictly positive
    beta ~ Dirichlet([0.5, 0.5])  # Prior for the base distribution

    # Small positive constant to ensure all values are positive
    epsilon = 1e-6

    # Container-specific color distributions
    theta = Vector{Vector{Real}}(undef, I)
    for i in 1:I
        concentration_param = alpha * beta .+ epsilon
        
        # Safeguard: check if concentration_param contains Inf or NaN values
        if any(isinf.(concentration_param)) || any(isnan.(concentration_param)) || any(concentration_param .<= 0)
            error("Invalid concentration parameters detected: ", concentration_param)
        end

        theta[i] ~ Dirichlet(concentration_param)

        # Ensure theta[i] values are above a minimum threshold
        min_threshold = 1e-6
        theta[i] = [max(val, min_threshold) for val in theta[i]]

        # Normalize theta[i] to make sure it sums to 1
        theta_sum = sum(theta[i])
        theta[i] = theta[i] / theta_sum

        # Ensure that theta[i] is a valid probability vector
        if any(isnan.(theta[i])) || any(theta[i] .< 0) || abs(sum(theta[i]) - 1.0) > epsilon
            error("Invalid probability vector detected: ", theta[i])
        end

        y[i, :] ~ Multinomial(n[i], theta[i])
    end
end


dirichlet_multinomial_model (generic function with 2 methods)

In [6]:
#Mixed group data
sample_vectors_mixed = [
    [0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],  
    [0, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1],  
    [0, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2], 
    [0, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2],  
    [0, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2], 
    [0, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2], 
    [0, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2],  
    [0, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2],  
    [0, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1], 
    [0, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2],  
    [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
]
#Uniform group data
sample_vectors_uniform = [
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  
    [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  
    [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  
    [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  
    [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 
    [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]  #Test
]


11-element Vector{Vector{Int64}}:
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
 [0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [11]:
# Function to fit the model incrementally using the subsets, plus an additional test2 container
function fit_dynamic_model_test(sample_vectors)
    subset_index = 1
    n_containers = length(sample_vectors)
    all_posteriors = []

    # Iterate through each container and build up the subsets
    for container_index in 1:length(sample_vectors) #For each container
        for sample_index in 1:length(sample_vectors[container_index]) #For each sample 
            # Build the current subset
            subset = [i < container_index ? sample_vectors[i] : sample_vectors[i][1:sample_index] for i in 1:container_index]

            # Prepare the counts (y) for the model
            I = length(subset)  # Number of containers in the subset
            K = 2  # Number of item types (2; Low and High value in this case)
            y = [zeros(Int, K) for _ in 1:I]  # Initialize y based on K

            # Populate y based on the current subset
            for i in 1:I
                for val in subset[i]
                    if val == 1
                        y[i][1] += 1  # First category (e.g., Low-value)
                    elseif val == 2
                        y[i][2] += 1  # Second category (e.g., High-value)
                    elseif val == 0
                        y = y #A zero indicates an empty draw
                    else
                        error("Invalid sample value: $val. Must be 0, 1 or 2.")
                    end
                end
            end

            # Prepare n based on the current subset
            n = [length(s)-1 for s in subset]  # Total counts for each container in the subset
            
            #If the current container is the test1 container (i.e. last container)
            if I == length(sample_vectors)
                push!(y, [0,0]) #Append a new container (test2), with no observation
                push!(n, 0) #Count items in new container (always 0)
                I = I+1 #Lengthen I by the additional container
            end
            
            # Print the current subset and the corresponding y element
            println("Subset $subset_index: $subset")
            println("y element for model: $y")
            println("I: $I")
            println("n: $n\n")

            # Fit the model with the current subset
            model = dirichlet_multinomial_model(K, I, n, y)
            chain = sample(model, NUTS(), 1000, progress = false)

            # Store the posterior samples in a dictionary
            posterior_samples = Dict()
            for param in names(chain)
                posterior_samples[string(param)] = chain[param]
            end
            push!(all_posteriors, posterior_samples)

            subset_index += 1
        end
    end
    
    return all_posteriors
end

fit_dynamic_model_test (generic function with 1 method)

In [13]:
# Fit the model with the sample_vectors
#all_posteriors_uniform = fit_dynamic_model_test(sample_vectors_uniform)
#all_posteriors_mixed = fit_dynamic_model_test(sample_vectors_mixed)
# Save the posteriors
#BSON.@save "Posterior_Samples/level2_mixed.bson" all_posteriors_mixed
#BSON.@save "Posterior_Samples/level2_uniform.bson" all_posteriors_uniform

# Load posteriror samples
BSON.@load "Posterior_Samples/level2_mixed.bson" all_posteriors_mixed
BSON.@load "Posterior_Samples/level2_uniform.bson" all_posteriors_uniform

In [17]:
posterior_samples_test1_mixed = extract_posterior_samples(all_posteriors_mixed, sample_vectors_mixed, "test1", 1, 2)
posterior_samples_test2_mixed = extract_posterior_samples(all_posteriors_mixed, sample_vectors_mixed, "test2", 1, 2)
posterior_samples_test1_uniform = extract_posterior_samples(all_posteriors_uniform, sample_vectors_uniform, "test1", 1, 2)
posterior_samples_test2_uniform = extract_posterior_samples(all_posteriors_uniform, sample_vectors_uniform, "test2", 1, 2)

#Save extracted samples
#BSON.@save "Posterior_Samples/level2_posterior_samples_test1_mixed.bson" posterior_samples_test1_mixed
#BSON.@save "Posterior_Samples/level2_posterior_samples_test2_mixed.bson" posterior_samples_test2_mixed
#BSON.@save "Posterior_Samples/level2_posterior_samples_test1_uniform.bson" posterior_samples_test1_uniform
#BSON.@save "Posterior_Samples/level2_posterior_samples_test2_uniform.bson" posterior_samples_test2_uniform